In [ ]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
  
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i : i + batch_size]


def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                                batch_size=16, device=device,
                                column_text='article',
                                column_summary='highlights'):
  article_batches = list(generate_batch_sized_chunks(dataset[column_text],batch_size))
  target_batches = list(generate_batch_sized_chunks(dataset[column_summary],batch_size))

  for article_batch, target_batch in tqdm(
      zip(article_batches, target_batches), total=len(article_batches)):

      inputs = tokenizer(article_batch, max_length=1024, truncation=True,
                         padding="max_length", return_tensors="pt")
      
      summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                        attention_mask=inputs["attention_mask"].to(device),
                        length_penalty=0.8, num_beams = 8, max_length=128)
      
      decoded_summaries = [tokenizer.decode(s,skip_special_tokens=True,
                                            clean_up_tokenization_spaces=True)
              for s in summaries]
       
      decoded_summaries = [d.replace(""," ") for d in decoded_summaries]

  
  
      metric.add_batch(predictions=decoded_summaries, references=target_batch)

      score = metric.compute()
      return score
  
  
      

      
      
  


In [ ]:
score = calculate_metric_on_test_ds(
    dataset_samsum['test'][0:10],rouge_metric,trainer.model,tokenizer,batch_size=2, device=device, column_text='dialogue', column_summary='summary'
)